In [86]:
# Conexão com SQL Server
import pandas as pd
import pyodbc

server = 'WILLIAN\SQLEXPRESS'
database = 'SQL_DB_1'

conexao = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                         f'SERVER={server};'
                         f'DATABASE={database};'
                         'Trusted_Connection=yes;')
cursor = conexao.cursor()


In [87]:
dados = pd.read_csv(r'C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Clientes.csv',delimiter=',')
dados.head()

,id,created_at,first_name,last_name,email,cell_phone,country,state,street,number,additionals
0,0,2017-11-01 14:45:41,Marta,Jesus,NaN,9 9102-7834,Brasil,Acre,NaN,NaN,Conjunto 16
1,1,2017-10-16 00:50:39,Luana,Almeida,NaN,9 7328-8718,Brasil,Rio Grande do Sul,Avenida 56 do Estado Rio Grande do Sul,989.0,Conjunto 17
2,2,2018-06-16 17:51:29,Frida,Mendes,frida@meu_email.com,9 5906-7552,Brasil,São Paulo,Avenida 59 do Estado São Paulo,534.0,NaN
3,3,2018-01-17 03:02:58,Daniela,Avelino,daniela@exemplo.com,9 4642-9486,Brasil,Mato Grosso,NaN,NaN,NaN
4,4,2018-08-06 07:24:16,Romário,Teixeira,NaN,9 3093-6522,Brasil,Bahia,Praça 56 do Estado Bahia,191.0,Apto 12


In [88]:
#Verificando os tipos de dados com forma de tabela
for coluna in dados.columns:
    print(f"Coluna: {coluna}: {dados[coluna].dtype}")

Coluna: id: int64
Coluna: created_at: object
Coluna: first_name: object
Coluna: last_name: object
Coluna: email: object
Coluna: cell_phone: object
Coluna: country: object
Coluna: state: object
Coluna: street: object
Coluna: number: float64
Coluna: additionals: object


In [89]:
#Converte coluna total_price para float64
dados['created_at'] = pd.to_datetime(dados['created_at'], errors='coerce') # - errors='coerce': diz ao pandas para forçar a conversão e, se encontrar algum valor inválido (como uma string malformada ou nula), ele vai substituir por NaT (Not a Time), que é o equivalente a NaN para datas.
# Se contrar valores NaT, você pode optar por removê-los ou preenchê-los com um valor padrão.
dados['email'] = dados['email'].fillna('Sem Registros')  # Se encontrar valores NaN, substitui por 'Sem Registros
dados['street'] = dados['street'].fillna('Sem Registros')
dados['number'] = dados['number'].fillna('Sem Registros')
dados['additionals'] = dados['additionals'].fillna('Sem Registros')

In [90]:
dados.head()

,id,created_at,first_name,last_name,email,cell_phone,country,state,street,number,additionals
0,0,2017-11-01 14:45:41,Marta,Jesus,Sem Registros,9 9102-7834,Brasil,Acre,Sem Registros,Sem Registros,Conjunto 16
1,1,2017-10-16 00:50:39,Luana,Almeida,Sem Registros,9 7328-8718,Brasil,Rio Grande do Sul,Avenida 56 do Estado Rio Grande do Sul,989.0,Conjunto 17
2,2,2018-06-16 17:51:29,Frida,Mendes,frida@meu_email.com,9 5906-7552,Brasil,São Paulo,Avenida 59 do Estado São Paulo,534.0,Sem Registros
3,3,2018-01-17 03:02:58,Daniela,Avelino,daniela@exemplo.com,9 4642-9486,Brasil,Mato Grosso,Sem Registros,Sem Registros,Sem Registros
4,4,2018-08-06 07:24:16,Romário,Teixeira,Sem Registros,9 3093-6522,Brasil,Bahia,Praça 56 do Estado Bahia,191.0,Apto 12


In [91]:
#Pegar o nome das colunas
colunas = dados.columns
str(colunas).replace("'","")

'Index([id, created_at, first_name, last_name, email, cell_phone,\n       country, state, street, number, additionals],\n      dtype=object)'

Apagar os dados antes de inserir

In [92]:
cursor.execute("TRUNCATE TABLE [Clientes]")  # Apagar os dados antes de inserir
cursor.commit()  # Validar dados no SQL Server

Inserindo DataFrame no Banco

In [93]:
for index, linha in dados.iterrows():
    linha.email = str(linha.email) # Convertendo para string pois tem caracteres especiais
    linha.state = str(linha.state)
    linha.street = str(linha.street)
    linha.number = str(linha.number)
    linha.additionals = str(linha.additionals)
    cursor.execute(
        "INSERT INTO [Clientes](id,created_at,first_name,last_name,email,cell_phone,country,state,street,number,additionals) VALUES (?,?,?,?,?,?,?,?,?,?,?)",
        linha.id,
        linha.created_at,
        linha.first_name,
        linha.last_name,
        linha.email,
        linha.cell_phone,
        linha.country,
        linha.state,
        linha.street,
        linha.number,
        linha.additionals)
cursor.commit()
cursor.close()
conexao.close()

In [94]:
print("Dados inseridos com sucesso na tabela Clientes.")

Dados inseridos com sucesso na tabela Clientes.
